# ADIÇÃO DE BIOMASSA MICROALGAL NO PROCESSO DE ELETROCOAGULAÇÃO/ FLOTAÇÃO PARA REMOÇÃO DE ÍONS CU2+ E NI2+ DE EFLUENTES GERADOS EM INDÚSTRIAS DE GALVANOPLASTIA

### Parte integrante da tese de Doutorado na Universidade Federal do Ceará - UFC
### Autora: Liana Geisa Conrado Maia

# Efluente real

## Importando bibliotecas

In [1]:
# !pip install sklearn
import pandas as pd
import numpy as np
from pandas import read_csv
import os
from sklearn import preprocessing
from pylab import *
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (20,20)

## Carregando o dataset

In [2]:
# Carregando os df
arquivo = 'efluente_real.csv'
df = read_csv(arquivo, sep="|")

FileNotFoundError: [Errno 2] File efluente_real.csv does not exist: 'efluente_real.csv'

## Explorando os df

In [ ]:
# Exibindo os df
df.head(5)

In [ ]:
# Tipos de df
df.dtypes

In [ ]:
# Resumo estatístico do dataframe
df.describe()

## Correlação entre variáveis

In [ ]:
cols = df.columns
cols = ['TDH', 'pH', 'Condutividade', 'Temperatura', 'Tensão',
       'Corrente', 'Turbidez', 'Consumo']

## Visualização Gráfica

In [ ]:
# Histograma
fig = plt.figure()
df[cols].hist()
plt.show()

In [ ]:
# Densidade
df[cols].plot(kind='density', 
           subplots = True, 
           layout = (3,3), 
           sharex = False)

plt.show()

In [ ]:
# Boxplot
fig = plt.figure()
df[cols].plot(kind = 'box', 
              subplots = True, 
              layout = (3,3), 
              sharey = False)
plt.tight_layout()
plt.show()

## Correlação entre variáveis

In [ ]:
df[cols].corr(method = 'pearson')

In [ ]:
cols = [
    'TDH', 
    'pH', 
    'Condutividade', 
    'Temperatura', 
    'Tensão',
    'Corrente', 
    'Turbidez', 
    'Consumo']

correlations = df[cols].corr()

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin = -1, vmax = 1)
fig.colorbar(cax)
ticks = np.arange(0, len(cols), 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(cols, rotation = 90)
ax.set_yticklabels(cols)
plt.show()

In [ ]:
import numpy as np
import statsmodels.api as sm


df_filt = df

y = df_filt['Consumo']
    
cols = [
    #'TDH', 
    'pH', 
    #'Condutividade', 
    'Temperatura', 
    'Tensão',
    #'Corrente', 
    #'Turbidez', 
    #'Consumo',
]

x = df_filt[cols]


# x = sm.add_constant(x)
mod = sm.OLS(y, x)
res = mod.fit()
print('Parameters: ')
print(res.params)
print('R2: ', res.rsquared)
res.summary()

In [ ]:
import plotly.express as px

fig = px.scatter_3d(
    df, 
    x='pH', 
    y='Temperatura', 
    z='Tensão',
    color='Consumo')
fig.show()

In [ ]:
import plotly.express as px

cols = [
    #'TDH', 
    'pH', 
    #'Condutividade', 
    'Temperatura', 
    'Tensão',
    #'Corrente', 
    #'Turbidez', 
    #'Consumo',
]

graphics = (
    ('pH', 'Tensão'),
    ('pH', 'Temperatura'),
    ('Tensão', 'Temperatura'),
)

for g in graphics:
    df_temp = df
    fig = px.line(df_temp, 
                  x=g[0], 
                  y=g[1], 
                  #color='Tensão',
                 title="% remoção de níquel por tempo de processo")
    fig.show()

## Selecionando a amostra

In [ ]:
# Selecionando amostra

# Testes: ['50%MA', '75%MA', '90%MA', '95%MA', '100%MA']
def marcar_amostra_selecionada_cu(obj):
    if obj['Teste'] in ['50%MA', '75%MA', '90%MA', '95%MA', '100%MA']:
        return 1
    else:
        return 0
    
df['AmostraSelecionadaCu'] = df.apply(marcar_amostra_selecionada_cu, axis=1)

# Testes: ['5%MA', '50%MA', '75%MA', '90%MA', '95%MA']

def marcar_amostra_selecionada_ni(obj):
    if obj['Teste'] in ['5%MA', '50%MA', '75%MA', '90%MA', '95%MA']:
        return 1
    else:
        return 0
    
df['AmostraSelecionadaNi'] = df.apply(marcar_amostra_selecionada_ni, axis=1)
    
# https://pt.stackoverflow.com/questions/340114/dataframe-pandas-calcular-coluna-com-base-em-outras


## Estudando o cobre (Cu)

In [ ]:
import plotly.express as px

df_temp = df.query("AmostraSelecionadaCu==1")
fig = px.line(df_temp, x="Tempo", y="Cu", color='Teste')
fig.show()

In [ ]:
import numpy as np
import statsmodels.api as sm

df_filt = df.query('AmostraSelecionadaCu==1').dropna()

y = df_filt['Cu']
    
cols = [
    'Tempo', 
    'Tensão', 
    'Corrente', 
    'pH',
]
x = df_filt[cols]

mod = sm.OLS(y, x)
res = mod.fit()
print('Parameters: ', res.params)
print('R2: ', res.rsquared)
res.summary()

## Estudando o Niquel (Ni)

In [ ]:
import plotly.express as px

df_temp = df.query("AmostraSelecionadaNi==1")
fig = px.line(df_temp, x="Tempo", y="Ni", color='Teste')
fig.show()

In [ ]:
import numpy as np
import statsmodels.api as sm

df_filt = df.dropna()

y = df_filt['Cu']

#'Turbidez',
#'Condutividade', 'DO680', 'Cu', 'Ni'
    
cols = [
    'Tempo', 
    'Tensão', 
    'Corrente', 
    'pH',
]

x = df_filt[cols]
mod = sm.OLS(y, x)
res = mod.fit()
print('Parameters: ', res.params)
print('R2: ', res.rsquared)
res.summary()

